### Import Required Packages and Set Options

#### Import Base Libraries

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import multiprocessing as mp

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from functools import partial

#### Put the Main Package Library on the PYTHONPATH

In [2]:
curdir = sys.path[0]
srcdir = os.path.join(os.path.split(curdir)[0], 'rankfm')
sys.path[0] = srcdir
srcdir

'/Users/ericlundquist/Repos/rankfm/rankfm'

#### Dynamically Re-Load all Package Modules on Execution

In [5]:
%load_ext autoreload
%autoreload 2

from rankfm import RankFM
from evaluation import precision_at_k, recall_at_k

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set File Path Constants

In [6]:
REPO_ROOT = os.path.split(srcdir)[0]
DATA_ROOT = os.path.join(REPO_ROOT, "data/ml-100k")
print("\n".join([REPO_ROOT, DATA_ROOT]))

/Users/ericlundquist/Repos/rankfm
/Users/ericlundquist/Repos/rankfm/data/ml-100k


### Prepare Example Data

#### Load Users Data

In [7]:
users_df = pd.read_csv(os.path.join(DATA_ROOT, "users.csv"))
users_df['agegroup'] = pd.cut(users_df['age'], [0, 30, 45, 100], right=False, labels=False)
users_df = users_df.drop(['age', 'zip_code'], axis=1)
users_df = pd.get_dummies(users_df, prefix_sep='__', columns=['agegroup', 'gender', 'occupation'])
users_df.mean()

user_id                      472.000000
agegroup__0                    0.433722
agegroup__1                    0.348887
agegroup__2                    0.217391
gender__F                      0.289502
gender__M                      0.710498
occupation__administrator      0.083775
occupation__artist             0.029692
occupation__doctor             0.007423
occupation__educator           0.100742
occupation__engineer           0.071050
occupation__entertainment      0.019088
occupation__executive          0.033934
occupation__healthcare         0.016967
occupation__homemaker          0.007423
occupation__lawyer             0.012725
occupation__librarian          0.054083
occupation__marketing          0.027572
occupation__none               0.009544
occupation__other              0.111347
occupation__programmer         0.069989
occupation__retired            0.014846
occupation__salesman           0.012725
occupation__scientist          0.032874
occupation__student            0.207847


#### Load Items Data

In [8]:
items_df = pd.read_csv(os.path.join(DATA_ROOT, "items.csv"))
item_names = items_df[['item_id', 'item_name']]
item_names.head()

,item_id,item_name
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [9]:
items_df = items_df.drop(['item_name', 'release_date'], axis=1)
items_df.columns = ['item_id'] + ["genre__{}".format(col) for col in items_df.columns[1:]]
items_df.mean()

item_id               841.500000
genre__action           0.149227
genre__adventure        0.080262
genre__animation        0.024970
genre__childrens        0.072533
genre__comedy           0.300238
genre__crime            0.064804
genre__documentary      0.029727
genre__drama            0.431034
genre__fantasy          0.013080
genre__film_noir        0.014269
genre__horror           0.054697
genre__musical          0.033294
genre__mystery          0.036266
genre__romance          0.146849
genre__scifi            0.060048
genre__thriller         0.149227
genre__war              0.042212
genre__western          0.016052
dtype: float64

#### Load Ratings Data

In [10]:
ratings_explicit = pd.read_csv(os.path.join(DATA_ROOT, "ratings.csv"))
ratings_explicit['timestamp'] = pd.to_datetime(ratings_explicit['unix_timestamp'], origin='unix', unit='s')
ratings_explicit['positive_feedback'] = ratings_explicit.groupby('user_id')['rating'].transform(lambda c: np.where(c > c.mean(), 1, 0))
ratings_explicit = ratings_explicit.drop('unix_timestamp', axis=1)
ratings_explicit.mean()

user_id              462.48475
item_id              425.53013
rating                 3.52986
positive_feedback      0.54194
dtype: float64

In [11]:
ratings_explicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,196,242,3,1997-12-04 15:55:49,0
1,186,302,3,1998-04-04 19:22:22,0
2,22,377,1,1997-11-07 07:18:36,0
3,244,51,2,1997-11-27 05:02:03,0
4,166,346,1,1998-02-02 05:33:16,0


#### Generate Implicit Feedback Ratings Data

In [12]:
ratings_implicit = ratings_explicit[ratings_explicit.positive_feedback == 1].reset_index(drop=True)
ratings_implicit.head()

,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


#### Print Final Matrix Shapes

In [13]:
print(ratings_explicit.user_id.nunique(), ratings_explicit.item_id.nunique())
print(ratings_implicit.user_id.nunique(), ratings_implicit.item_id.nunique())

print(ratings_explicit.shape)
print(ratings_implicit.shape)

943 1682
943 1483
(100000, 5)
(54194, 5)


#### Create (Training, Validation) Interactions for Evaluation

In [14]:
print(ratings_implicit.shape)
ratings_implicit.head()

(54194, 5)


,user_id,item_id,rating,timestamp,positive_feedback
0,253,465,5,1998-04-03 18:34:27,1
1,286,1014,5,1997-11-17 15:38:45,1
2,200,222,5,1997-10-05 09:05:40,1
3,224,29,3,1998-02-21 23:40:57,1
4,122,387,5,1997-11-11 17:47:39,1


In [15]:
test_pct = 0.25
drop_users = (2, 4)

train_mask = (ratings_implicit.index / ratings_implicit.shape[0]) <  (1 - test_pct)
valid_mask = (ratings_implicit.index / ratings_implicit.shape[0]) >= (1 - test_pct)

interactions_train = ratings_implicit[train_mask][['user_id', 'item_id']]
interactions_valid = ratings_implicit[valid_mask][['user_id', 'item_id']]

# drop some users from the training data to test cold-start functionality
interactions_train = interactions_train[~interactions_train.user_id.isin(drop_users)]

train_users = np.sort(interactions_train.user_id.unique())
valid_users = np.sort(interactions_valid.user_id.unique())
cold_start_users = set(valid_users) - set(train_users)

print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))
print("train users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
print("cold-start users: {}".format(cold_start_users))

train shape: (40598, 2)
valid shape: (13548, 2)
train users: 941
valid users: 905
cold-start users: {2, 4}


### Test Out Core Package Functionality

#### Initialize the Model

In [16]:
model = RankFM(factors=10, learning_rate=0.1, regularization=0.01, sigma=0.1)
model

#### Fit the Model on the Training Data and Profile Computational Performance

In [18]:
interactions = ratings_implicit[['user_id', 'item_id']]
interactions.shape

(54194, 2)

In [19]:
%%prun -l 10 

model.fit(interactions, epochs=10, verbose=True)


training epoch: 0
penalized log-likelihood: -19536.8

training epoch: 1
penalized log-likelihood: -18320.77

training epoch: 2
penalized log-likelihood: -16898.42

training epoch: 3
penalized log-likelihood: -14639.53

training epoch: 4
penalized log-likelihood: -12968.07

training epoch: 5
penalized log-likelihood: -11785.26

training epoch: 6
penalized log-likelihood: -10956.02

training epoch: 7
penalized log-likelihood: -10305.61

training epoch: 8
penalized log-likelihood: -9688.43

training epoch: 9
penalized log-likelihood: -9444.41
 

         3058379 function calls (3057146 primitive calls) in 38.158 seconds

   Ordered by: internal time
   List reduced from 762 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1352   18.556    0.014   18.556    0.014 {method 'acquire' of '_thread.lock' objects}
   541940   12.773    0.000   14.632    0.000 rankfm.py:144(_gradient_step)
  1083880    2.440    0.000    3.398    0.000 rankfm.py:132(_pairwise_utility)
       10    2.005    0.201    3.545    0.354 rankfm.py:181(<listcomp>)
  1083880    0.957    0.000    0.957    0.000 {built-in method numpy.dot}
        1    0.409    0.409   38.154   38.154 rankfm.py:191(fit)
      360    0.202    0.001    0.202    0.001 {built-in method posix.waitpid}
       10    0.180    0.018    0.184    0.018 pool.py:397(_map_async)
       80    0.104    0.001    0.129    0.002 {built-in method posix.fork}
       31    0.043    0.001    0.043    0.001 {built-in method builtins.sum}

#### Evaluate Model Performance on the Validation Data

In [20]:
interactions_valid.head()

,user_id,item_id
40646,437,443
40647,712,1037
40648,903,1070
40649,151,629
40650,711,79


In [21]:
k = 10

In [22]:
# %%prun -l 10 

model_pre = round(precision_at_k(model, interactions_valid, k=k), 3)
model_rec = round(recall_at_k(model, interactions_valid, k=k), 3)
print("model precision: {} model recall: {}".format(model_pre, model_rec))

model precision: 0.141 model recall: 0.15


#### Evaluate Pure Popularity Baseline

In [23]:
most_popular = interactions_train.groupby('item_id').size().sort_values(ascending=False)[:k]
most_popular

item_id
50     357
100    291
181    277
258    260
98     249
174    246
127    245
1      225
56     223
313    221
dtype: int64

In [24]:
test_user_items = interactions_valid.groupby('user_id')['item_id'].apply(set).to_dict()
test_recommends = model.recommend_for_users(list(test_user_items.keys()), cold_start='drop')
test_user_items = {key: val for key, val in test_user_items.items() if key in test_recommends.index}

base_pre = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()]), 3)
base_rec = round(np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()]), 3)

print("number of test users: {}".format(len(test_user_items)))
print("baseline precision: {} baseline recall: {}".format(base_pre, base_rec))

number of test users: 905
baseline precision: 0.098 baseline recall: 0.087


#### Generate Model Scores for Test User/Items

In [25]:
mask = (interactions_valid.user_id.isin(interactions_train.user_id)) & (interactions_valid.item_id.isin(interactions_train.item_id))
print(interactions_valid.shape)
print(interactions_valid[mask].shape)

(13548, 2)
(13469, 2)


In [26]:
scores_miss = model.predict(interactions_valid, cold_start='nan')
scores_drop = model.predict(interactions_valid, cold_start='drop')

print(scores_miss.shape)
print(scores_drop.shape)

(13548,)
(13548,)


In [27]:
print(scores_miss.mean(), scores_miss.std(), scores_miss.min(), scores_miss.max())
print(scores_drop.mean(), scores_drop.std(), scores_drop.min(), scores_drop.max())

1.8329248608726232 1.0077090765407475 -2.309359856555897 5.050364960587583
1.8329248608726232 1.0077090765407475 -2.309359856555897 5.050364960587583


#### Spot-Check Some User Recommendations

In [28]:
len(train_users), len(valid_users), cold_start_users

(941, 905, {2, 4})

In [29]:
recommendations = model.recommend_for_users(valid_users, n_items=10, filter_previous=True, cold_start='nan')
print(recommendations.shape)
recommendations.head(10)

(905, 10)


,0,1,2,3,4,5,6,7,8,9
1,56,50,12,98,174,475,238,474,175,191
2,286,302,258,14,690,306,289,300,9,276
3,333,690,270,300,301,682,289,331,294,748
4,690,302,301,333,289,300,270,310,347,258
5,172,210,151,144,173,181,91,174,176,98
6,483,474,480,357,197,191,514,603,132,504
7,98,174,176,132,172,22,204,483,429,64
8,98,172,22,50,174,176,64,79,195,144
9,50,258,127,98,100,181,483,515,56,191
10,474,480,483,514,197,357,190,478,511,603


In [30]:
recommendations[recommendations.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9


In [31]:
user_id = 6
user_recs = recommendations.loc[user_id]
user_item_names = item_names[item_names.item_id.isin(user_recs)].set_index('item_id').loc[user_recs]
user_item_names

,item_name
item_id,
483,Casablanca (1942)
474,Dr. Strangelove or: How I Learned to Stop Worr...
480,North by Northwest (1959)
357,One Flew Over the Cuckoo's Nest (1975)
197,"Graduate, The (1967)"
191,Amadeus (1984)
514,Annie Hall (1977)
603,Rear Window (1954)
132,"Wizard of Oz, The (1939)"


### Look at the Movies with the Highest/Lowest Model Weights

In [32]:
item_weights = pd.DataFrame({'item_idx': np.arange(len(model.w_item)), 'w_item': model.w_item})
item_weights['item_id'] = item_weights['item_idx'].map(model.index_to_item)
item_weights = pd.merge(item_weights, item_names, on='item_id').sort_values('w_item', ascending=False)
item_weights = item_weights[['item_id', 'item_idx', 'item_name', 'w_item']]

#### Best Movies Ever?

In [33]:
item_weights.head(10)

,item_id,item_idx,item_name,w_item
49,50,49,Star Wars (1977),2.537779
255,258,255,Contact (1997),2.310012
178,181,178,Return of the Jedi (1983),2.294143
99,100,99,Fargo (1996),2.185690
124,127,124,"Godfather, The (1972)",2.164586
297,300,297,Air Force One (1997),2.125126
97,98,97,"Silence of the Lambs, The (1991)",2.113337
285,288,285,Scream (1996),2.098725
148,151,148,Willy Wonka and the Chocolate Factory (1971),2.058254
6,7,6,Twelve Monkeys (1995),2.033065


#### Worst Movies Ever?

In [34]:
item_weights.tail(10)

,item_id,item_idx,item_name,w_item
35,36,35,Mad Love (1995),-1.763544
1458,1633,1458,¡ kˆldum klaka (Cold Fever) (1994),-1.766717
1162,1201,1162,Marlene Dietrich: Shadow and Light (1996),-1.776563
1262,1319,1262,"Neon Bible, The (1995)",-1.776729
1321,1410,1321,Harlem (1993),-1.784217
1277,1351,1277,Lover's Knot (1996),-1.785110
1353,1458,1353,"Damsel in Distress, A (1937)",-1.791392
804,822,804,Faces (1968),-1.809831
1185,1225,1185,Angus (1995),-1.836384
1378,1490,1378,Fausto (1993),-1.838051


# Start Sandbox Code

In [36]:
test = pd.DataFrame(np.arange(9).reshape(3, 3))
test

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8


In [40]:
test.columns = ['user_id'] + [str(col) for col in test.columns[1:]]
test

,user_id,1,2
0,0,1,2
1,3,4,5
2,6,7,8


In [39]:
test.columns[1:]

RangeIndex(start=1, stop=3, step=1)

In [42]:
test = pd.DataFrame(np.zeros(100), columns=['user_id'])
print(test.shape)
test.head()

(100, 1)


,user_id
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [47]:
test = np.zeros((100, 1))
test[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [49]:
np.float32

numpy.float32

In [51]:
np.arange(9).reshape(3, 3).astype(np.float32)

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]], dtype=float32)